In [1]:
import argparse
import collections

import numpy as np

import torch
import torch.optim as optim
from torchvision import transforms
import sys
sys.path.append('../')

from retinanet import model
from retinanet.dataloader import CocoDataset, CSVDataset, collater, Resizer, AspectRatioBasedSampler, Augmenter, \
    Normalizer
from torch.utils.data import DataLoader

from retinanet import coco_eval
from retinanet import csv_eval


In [2]:
coco_path = '../COCO'
dataset_train = CocoDataset(coco_path, set_name='train2017',
                            transform=transforms.Compose([Normalizer(), Augmenter(), Resizer()]))
dataset_val = CocoDataset(coco_path, set_name='val2017',
                          transform=transforms.Compose([Normalizer(), Resizer()]))
dataset_train.num_classes()

loading annotations into memory...
Done (t=18.59s)
creating index...
index created!
loading annotations into memory...
Done (t=0.49s)
creating index...
index created!


80

In [3]:
sampler = AspectRatioBasedSampler(dataset_train, batch_size=32, drop_last=False)
dataloader_train = DataLoader(dataset_train, num_workers=3, collate_fn=collater, batch_sampler=sampler)
sampler_val = AspectRatioBasedSampler(dataset_val, batch_size=1, drop_last=False)
dataloader_val = DataLoader(dataset_val, num_workers=3, collate_fn=collater, batch_sampler=sampler_val)



In [4]:
# for i in dataloader_train :
#     print(i)
#     adfsf

In [5]:
torch.cuda.is_available()

True

In [6]:
%time
PATH_TO_WEIGHTS = '../coco_resnet_50_map_0_335_state_dict.pt'
retinanet = model.resnet50(num_classes=dataset_train.num_classes(),)
retinanet.load_state_dict(torch.load(PATH_TO_WEIGHTS))

CPU times: user 8 µs, sys: 4 µs, total: 12 µs
Wall time: 29.1 µs


/home/beomgon/anaconda3/envs/torch_retina/lib/python3.7/site-packages/torch/cuda/__init__.py:104: UserWarning: 
GeForce RTX 3090 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 compute_37.
If you want to use the GeForce RTX 3090 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


<All keys matched successfully>

In [7]:
# retinanet.anchors()

In [8]:
device = torch.device('cuda')
# device = torch.device('cpu')
retinanet.to(device)
retinanet = torch.nn.DataParallel(retinanet).to(device)

In [9]:
retinanet.training = True
optimizer = optim.Adam(retinanet.parameters(), lr=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)
loss_hist = collections.deque(maxlen=500)
retinanet.train()
# retinanet.module.freeze_bn()
print('Num training images: {}'.format(len(dataset_train)))


Num training images: 118287


In [10]:
for epoch_num in range(10):

    retinanet.train()
#     retinanet.module.freeze_bn()
    epoch_loss = []
    print('**************************************', epoch_num )
    for iter_num, data in enumerate(dataloader_train):
        print(type(data))
        try:
            optimizer.zero_grad()

            if torch.cuda.is_available():
                classification_loss, regression_loss = retinanet([data['img'].cuda().float(), data['annot']])
            else:
                classification_loss, regression_loss = retinanet([data['img'].float(), data['annot']])
            print('data load is done')

            classification_loss = classification_loss.mean()
            regression_loss = regression_loss.mean()
            loss = classification_loss + regression_loss

            if bool(loss == 0):
                continue

            loss.backward()
            torch.nn.utils.clip_grad_norm_(retinanet.parameters(), 0.1)
            optimizer.step()
            loss_hist.append(float(loss))
            epoch_loss.append(float(loss))

            print(
                'Epoch: {} | Iteration: {} | Classification loss: {:1.5f} | Regression loss: {:1.5f} | Running loss: {:1.5f}'.format(
                    epoch_num, iter_num, float(classification_loss), float(regression_loss), np.mean(loss_hist)))

            del classification_loss
            del regression_loss
        except Exception as e:
            print(e)
            continue

    print('Evaluating dataset')
    coco_eval.evaluate_coco(dataset_val, retinanet)
    scheduler.step(np.mean(epoch_loss))
    torch.save(retinanet.module, '{}_retinanet_{}.pt'.format(parser.dataset, epoch_num))

retinanet.eval()
torch.save(retinanet, 'model_final.pt')

************************************** 0
<class 'dict'>
CUDA error: out of memory
<class 'dict'>
CUDA error: out of memory
<class 'dict'>
CUDA error: out of memory
<class 'dict'>
CUDA error: out of memory
<class 'dict'>
CUDA error: out of memory
<class 'dict'>
CUDA error: out of memory
<class 'dict'>
CUDA error: out of memory
<class 'dict'>
CUDA error: out of memory
<class 'dict'>
CUDA error: out of memory
<class 'dict'>
CUDA error: out of memory
<class 'dict'>
CUDA error: out of memory
<class 'dict'>
CUDA error: out of memory
<class 'dict'>
CUDA error: out of memory
<class 'dict'>
CUDA error: out of memory
<class 'dict'>
CUDA error: out of memory
<class 'dict'>
CUDA error: out of memory
<class 'dict'>
CUDA error: out of memory
<class 'dict'>
CUDA error: out of memory
<class 'dict'>
CUDA error: out of memory
<class 'dict'>
CUDA error: out of memory
<class 'dict'>
CUDA error: out of memory
<class 'dict'>
CUDA error: out of memory
<class 'dict'>
CUDA error: out of memory
<class 'dict'>
C

RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/beomgon/anaconda3/envs/torch_retina/lib/python3.7/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/home/beomgon/anaconda3/envs/torch_retina/lib/python3.7/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "../retinanet/model.py", line 240, in forward
    x = self.relu(x)
  File "/home/beomgon/anaconda3/envs/torch_retina/lib/python3.7/site-packages/torch/nn/modules/module.py", line 889, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/home/beomgon/anaconda3/envs/torch_retina/lib/python3.7/site-packages/torch/nn/modules/activation.py", line 102, in forward
    return F.relu(input, inplace=self.inplace)
  File "/home/beomgon/anaconda3/envs/torch_retina/lib/python3.7/site-packages/torch/nn/functional.py", line 1204, in re